In [5]:
import pandas as pd
from transformers import pipeline
import ray
from ray import serve
from fastapi import FastAPI
from pydantic import PositiveInt, constr, BaseModel
import requests
from typing import List
from transformers import pipeline
import gradio as gr
import matplotlib.pyplot as plt
from ray.serve.gradio_integrations import GradioServer

In [6]:
df = pd.read_csv('films_prepared_v2.csv')
df

,movieID,chrono,characters,replics,names,movie_title,genres
0,m0,"['L194', 'L195', 'L196', 'L197']","['u0', 'u2', 'u0', 'u2']",['Can we make this quick? Roxanne Korrine and...,"['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
1,m0,"['L198', 'L199']","['u0', 'u2']","[""You're asking me out. That's so cute. What'...","['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
2,m0,"['L200', 'L201', 'L202', 'L203']","['u0', 'u2', 'u0', 'u2']","[""No no it's my fault -- we didn't have a prop...","['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
3,m0,"['L204', 'L205', 'L206']","['u2', 'u0', 'u2']","['Why?', 'Unsolved mystery. She used to be re...","['CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
4,m0,"['L207', 'L208']","['u0', 'u2']",['Gosh if only we could find Kat a boyfriend.....,"['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
...,...,...,...,...,...,...,...
83092,m616,"['L666324', 'L666325', 'L666326', 'L666327']","['u9028', 'u9031', 'u9028', 'u9031']",['Do you think she might be interested in som...,"['COGHILL', 'MELVILL', 'COGHILL', 'MELVILL']",zulu dawn,"['action', 'adventure', 'drama', 'history', 'w..."
83093,m616,"['L666575', 'L666576']","['u9028', 'u9031']","[""Choose your targets men. That's right Watch ...","['COGHILL', 'MELVILL']",zulu dawn,"['action', 'adventure', 'drama', 'history', 'w..."
83094,m616,"['L666256', 'L666257']","['u9034', 'u9030']","[""Colonel Durnford... William Vereker. I hear ...","['VEREKER', 'DURNFORD']",zulu dawn,"['action', 'adventure', 'drama', 'history', 'w..."
83095,m616,"['L666369', 'L666370', 'L666371', 'L666372']","['u9030', 'u9034', 'u9030', 'u9034']","['Your orders Mr Vereker?', ""I'm to take the S...","['DURNFORD', 'VEREKER', 'DURNFORD', 'VEREKER']",zulu dawn,"['action', 'adventure', 'drama', 'history', 'w..."


In [7]:
!ray start --head

2023-06-16 18:25:23,270	INFO usage_lib.py:408 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.
2023-06-16 18:25:23,270	INFO scripts.py:712 -- Local node IP: 127.0.0.1
2023-06-16 18:25:28,953	SUCC scripts.py:749 -- --------------------
2023-06-16 18:25:28,953	SUCC scripts.py:750 -- Ray runtime started.
2023-06-16 18:25:28,953	SUCC scripts.py:751 -- --------------------
2023-06-16 18:25:28,953	INFO scripts.py:753 -- Next steps
2023-06-16 18:25:28,953	INFO scripts.py:768 -- To connect to this Ray cluster:
2023-06-16 18:25:28,961	INFO scripts.py:770 -- import ray
2023-06-16 18:25:28,961	INFO scripts.py:771 -- ray.init()
2023-06-16 18:25:28,961	INFO scripts.py:783 -- 

In [8]:
@serve.deployment(route_prefix="/sentiment", name="sentiment")
class SentimentDeployment:
    def __init__(self):
        self.classifier = pipeline("sentiment-analysis",
                                   model="finiteautomata/bertweet-base-sentiment-analysis")

    async def __call__(self, request):
        data = await request.body()
        [result] = self.classifier(str(data))
        return result["label"]

In [9]:
ray.init(address='auto', namespace="serve-example", ignore_reinit_error=True)
serve.start(detached=True)

SentimentDeployment.deploy()

2023-06-16 18:25:36,797	INFO worker.py:1452 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2023-06-16 18:25:36,848	INFO worker.py:1627 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
(ServeController pid=9740) INFO 2023-06-16 18:25:49,799 controller 9740 deployment_state.py:1298 - Deploying new version of deployment sentiment.
(HTTPProxyActor pid=8320) INFO:     Started server process [8320]
(ServeController pid=9740) INFO 2023-06-16 18:25:49,878 controller 9740 deployment_state.py:1537 - Adding 1 replica to deployment sentiment.
(ServeReplica:sentiment pid=12256) emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [11]:
%%capture
def emotio(texts):
    texts = eval(texts)
    speaker_1 = [k for i, k in enumerate(texts) if not i%2]
    speaker_2 = [i for i in texts if i not in speaker_1]
    speaker_1_ems = {}
    speaker_2_ems = {}
    for i in speaker_1:
        result = requests.get("http://127.0.0.1:8000/sentiment", data=i).text
        speaker_1_ems[i] = result
    for i in speaker_2:
        result = requests.get("http://127.0.0.1:8000/sentiment", data=i).text
        speaker_2_ems[i] = result
    df_sp1 = pd.DataFrame({'Speaker': 'Speaker_1',
                       'Text': list(speaker_1_ems.keys()),
                       'Emotion': list(speaker_1_ems.values())})
    df_sp2 = pd.DataFrame({'Speaker': 'Speaker_2',
                       'Text': list(speaker_2_ems.keys()),
                       'Emotion': list(speaker_2_ems.values())})
    emot = pd.concat([df_sp1, df_sp2]).groupby('Speaker')['Emotion'].value_counts(normalize=True)
    
    
    fig = plt.figure()

    total_axes = 2
    subplt = 1 # номер начального axes
    for x in emot.index.get_level_values('Speaker').unique():
        ax = fig.add_subplot(1, total_axes, subplt) # добавляем axes для каждого графика
        plt.pie(emot[x], labels=emot[x].index.tolist(), autopct='%1.1f%%')
        plt.xlabel(x)
        subplt+=1

    
    # plt.show()
    
    return fig


In [14]:
%%capture
def gradio_builder():
    theme = gr.themes.Monochrome(primary_hue="blue", secondary_hue="blue").set(
    button_primary_background_fill="#FF0000",
    button_primary_background_fill_hover="*button_primary_background_fill",
                                )
    with gr.Blocks(theme=theme) as demo:
        with gr.Row():
            text = gr.Textbox(label="Input prompt")
            btn = gr.Button(value="Получить статистику по эмоциям")
        with gr.Row():
            with gr.Column(scale=1, min_width=400):
                btn.click(emotio, inputs=text, outputs=gr.Plot(label='Статистика'))
    return demo

In [15]:

serve.run(
    GradioServer.options(ray_actor_options={"num_cpus": 1}).bind(
        gradio_builder
    )
)

The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
(ServeController pid=9740) INFO 2023-06-16 18:27:25,796 controller 9740 deployment_state.py:1298 - Deploying new version of deployment default_GradioIngress.
(ServeController pid=9740) INFO 2023-06-16 18:27:25,882 controller 9740 deployment_state.py:1537 - Adding 1 replica to deployment default_GradioIngress.


RayServeSyncHandle(deployment='default_GradioIngress')

(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:40,259 default_GradioIngress default_GradioIngress#solMZF wPiiHsOxCc / default replica.py:654 - __CALL__ OK 125.5ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:40,427 default_GradioIngress default_GradioIngress#solMZF oBmzXfRTDf /info default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:40,443 default_GradioIngress default_GradioIngress#solMZF sbOjmRpJxF /theme.css default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:40,627 default_GradioIngress default_GradioIngress#solMZF BsDWMOcfbV /app_id default replica.py:654 - __CALL__ OK 8.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:42,896 default_GradioIngress default_GradioIngress#solMZF hKzQokFGIi /app_id default replica.py:654 - __CALL__ OK 8.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 1

(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:53,119 default_GradioIngress default_GradioIngress#solMZF kUIFhDuMvP /app_id default replica.py:654 - __CALL__ OK 8.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:53,388 default_GradioIngress default_GradioIngress#solMZF NwclMdUIkn /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:53,666 default_GradioIngress default_GradioIngress#solMZF PyjKqUGjrI /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:sentiment pid=12256) INFO 2023-06-16 18:27:53,948 sentiment sentiment#lglrBg ymrHzihpLm /sentiment replica.py:654 - __CALL__ OK 350.1ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:27:53,948 default_GradioIngress default_GradioIngress#solMZF vdimeXfevj /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:sentiment pid=12256) INFO 2023-06-16 18:27:54,182 sentiment sentiment#lglrBg qBLNMow

(ServeReplica:sentiment pid=12256) INFO 2023-06-16 18:28:05,298 sentiment sentiment#lglrBg RzoeLNDdzj /sentiment replica.py:654 - __CALL__ OK 221.8ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:05,246 default_GradioIngress default_GradioIngress#solMZF tIDAckHtWb /app_id default replica.py:654 - __CALL__ OK 15.3ms
(ServeReplica:sentiment pid=12256) INFO 2023-06-16 18:28:05,565 sentiment sentiment#lglrBg mSxhdxIwTs /sentiment replica.py:654 - __CALL__ OK 217.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:05,514 default_GradioIngress default_GradioIngress#solMZF yQBKhZDEOC /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:sentiment pid=12256) INFO 2023-06-16 18:28:05,798 sentiment sentiment#lglrBg maJPaMzwUh /sentiment replica.py:654 - __CALL__ OK 216.4ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:05,798 default_GradioIngress default_GradioIngress#solMZF gXTuZgTczK /app_id default replica.py:654 - __C

(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:15,711 default_GradioIngress default_GradioIngress#solMZF ZQAVZHsXqA /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:16,000 default_GradioIngress default_GradioIngress#solMZF JobdluFIts /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:16,298 default_GradioIngress default_GradioIngress#solMZF PjDeMMLoni /app_id default replica.py:654 - __CALL__ OK 4.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:16,568 default_GradioIngress default_GradioIngress#solMZF wUNiAsbVjq /app_id default replica.py:654 - __CALL__ OK 3.3ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-16 18:28:16,828 default_GradioIngress default_GradioIngress#solMZF QLhOPkLKuY /app_id default replica.py:654 - __CALL__ OK 0.0ms
(ServeReplica:default_GradioIngress pid=7380) INFO 2023-06-1

In [ ]:
#http://localhost:8000

In [16]:
!ray stop --force 

Exception in thread 2023-06-16 18:28:28,722	ERROR import_thread.py:80 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:6379 {created_time:"2023-06-16T15:28:28.689230103+00:00", grpc_status:2, grpc_message:"Stream removed"}"
>
ray_listen_error_messages:
Traceback (most recent call last):
  File "C:\Users\mosto\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
2023-06-16 18:28:28,722	ERROR worker.py:914 -- print_logs: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:6379 {created_time:"2023-06-16T15:28:28.72508958+00:00", grpc_status:2, grpc_message:"Stream removed"}"
>
    self.run()
  File "C:\Users\mosto\AppData\Local\Programs\Python\Python39\lib\threading.py", line 9

2023-06-16 18:28:28,206	VINFO scripts.py:1052 -- Killed `C:\Users\mosto\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\core\src\ray\raylet\raylet.exe --raylet_socket_name=tcp://127.0.0.1:56039 --store_socket_name=tcp://127.0.0.1:56407 --object_manager_port=0 --min_worker_port=10002 --max_worker_port=19999 --node_manager_port=0 --node_ip_address=127.0.0.1 --maximum_startup_concurrency=8 --static_resource_list=node:127.0.0.1,1.0,CPU,8,memory,1829732352,object_store_memory,914866176 "--python_worker_command=C:\Users\mosto\AppData\Local\Programs\Python\Python39\python.exe C:\Users\mosto\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\workers\setup_worker.py C:\Users\mosto\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\workers\default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=RAY_NODE_MANAGER_PORT_PLACEHOLDER --object-store-name=tcp://127.0.0.1:56407 --raylet-name=tcp://127.0.0.1:56039 --redis-address=None --temp-dir